# Cuaderno 3: Modelado Supervisado y Recomendación Clínica

**Tema:** Recommender Systems Clustering using Bayesian Non-Negative Matrix Factorization for Personalized Clinical Management in Critical Care

---

## Diagrama de Flujo

```
1. Carga de datos y embeddings latentes (BNMF/NMF)
   ↓
2. Preparación de variables para clasificación/regresión
   ↓
3. Entrenamiento de modelos supervisados (k-NN, MLP)
   ↓
4. Validación y métricas clínicas
   ↓
5. Sistema de recomendación basado en similitud latente
   ↓
6. Visualizaciones clínicas y tabla comparativa
```

## 1. Carga de Datos y Embeddings Latentes
Se cargan los datos preprocesados, los embeddings latentes (W de BNMF/NMF), y los transformadores necesarios.

In [ ]:
import pandas as pd
import numpy as np
import pickle

df = pd.read_csv('processed_data/data_prepared.csv')
try:
    with open('processed_data/bnmf_W.pkl', 'rb') as f:
        W_bnmf = pickle.load(f)
except:
    W_bnmf = None
try:
    with open('processed_data/nmf_W.pkl', 'rb') as f:
        W_nmf = pickle.load(f)
except:
    W_nmf = None

## 2. Preparación de Variables para Clasificación y Regresión
Se define la variable de riesgo (e.g., mortalidad) y una variable continua (e.g., score latente) para regresión.

In [ ]:
# Variable de clasificación: mortalidad hospitalaria
y_class = df['hospital_expire_flag']

# Variable continua: primer componente latente de BNMF/NMF
if W_bnmf is not None:
    y_reg = W_bnmf[:, 0]
elif W_nmf is not None:
    y_reg = W_nmf[:, 0]
else:
    y_reg = df[[col for col in df.columns if col not in ['hospital_expire_flag', 'data_split']]].sum(axis=1)

## 3. Entrenamiento de Modelos Supervisados
Incluye k-NN y MLP para clasificación y regresión, con validación cruzada y búsqueda de hiperparámetros.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor

# Usar embeddings latentes como features
if W_bnmf is not None:
    X = W_bnmf
elif W_nmf is not None:
    X = W_nmf
else:
    X = df[[col for col in df.columns if col not in ['hospital_expire_flag', 'data_split']]].values

X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.2, random_state=42, stratify=y_class)

# k-NN Classifier
knn_clf = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': [3, 5, 7]}, cv=5, scoring='accuracy')
knn_clf.fit(X_train, y_train)

# MLP Classifier
mlp_clf = GridSearchCV(MLPClassifier(max_iter=200, random_state=42), {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'learning_rate_init': [0.001, 0.01],
    'batch_size': [32, 64]
}, cv=3, scoring='accuracy')
mlp_clf.fit(X_train, y_train)

# Regresión
y_reg_train, y_reg_test = train_test_split(y_reg, test_size=0.2, random_state=42)
knn_reg = GridSearchCV(KNeighborsRegressor(), {'n_neighbors': [3, 5, 7]}, cv=5, scoring='neg_mean_squared_error')
knn_reg.fit(X_train, y_reg_train)
mlp_reg = GridSearchCV(MLPRegressor(max_iter=200, random_state=42), {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'learning_rate_init': [0.001, 0.01],
    'batch_size': [32, 64]
}, cv=3, scoring='neg_mean_squared_error')
mlp_reg.fit(X_train, y_reg_train)

## 4. Validación y Métricas Clínicas
Incluye métricas de clasificación (accuracy, precision, recall, F1, AUC-ROC) y regresión (RMSE, MAE, R²), matriz de confusión y curvas de aprendizaje.

In [ ]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
                             mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, ConfusionMatrixDisplay)
import matplotlib.pyplot as plt

# Clasificación
y_pred_knn = knn_clf.predict(X_test)
y_pred_mlp = mlp_clf.predict(X_test)

print('k-NN Classifier:')
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('Precision:', precision_score(y_test, y_pred_knn))
print('Recall:', recall_score(y_test, y_pred_knn))
print('F1:', f1_score(y_test, y_pred_knn))
print('AUC-ROC:', roc_auc_score(y_test, y_pred_knn))
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_knn)).plot()
plt.title('Matriz de Confusión k-NN')
plt.show()

print('MLP Classifier:')
print('Accuracy:', accuracy_score(y_test, y_pred_mlp))
print('Precision:', precision_score(y_test, y_pred_mlp))
print('Recall:', recall_score(y_test, y_pred_mlp))
print('F1:', f1_score(y_test, y_pred_mlp))
print('AUC-ROC:', roc_auc_score(y_test, y_pred_mlp))
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_mlp)).plot()
plt.title('Matriz de Confusión MLP')
plt.show()

# Regresión
y_pred_knn_reg = knn_reg.predict(X_test)
y_pred_mlp_reg = mlp_reg.predict(X_test)

print('k-NN Regressor:')
print('RMSE:', mean_squared_error(y_reg_test, y_pred_knn_reg, squared=False))
print('MAE:', mean_absolute_error(y_reg_test, y_pred_knn_reg))
print('R2:', r2_score(y_reg_test, y_pred_knn_reg))

print('MLP Regressor:')
print('RMSE:', mean_squared_error(y_reg_test, y_pred_mlp_reg, squared=False))
print('MAE:', mean_absolute_error(y_reg_test, y_pred_mlp_reg))
print('R2:', r2_score(y_reg_test, y_pred_mlp_reg))

## 5. Sistema de Recomendación Basado en Similitud Latente
Ejemplo de recomendación clínica: dado un paciente, sugerir tratamientos/procedimientos frecuentes en sus vecinos latentes.

In [ ]:
# Similitud de coseno entre pacientes en el espacio latente
from sklearn.metrics.pairwise import cosine_similarity

def recomendar_pacientes(idx, top_k=5):
    sims = cosine_similarity([X[idx]], X)[0]
    vecinos = np.argsort(sims)[-top_k-1:-1][::-1]
    print(f'Paciente {idx} - Vecinos más similares:', vecinos)
    # Aquí puedes mostrar tratamientos/eventos frecuentes de esos vecinos usando df original
# Ejemplo: recomendar para el primer paciente del test
recomendar_pacientes(X_test.index[0])

## 6. Visualizaciones Clínicas y Tabla Comparativa
Incluye mapas de calor de reconstrucción, proyecciones latentes y tabla de métricas.

In [ ]:
# Tabla comparativa de modelos
results = pd.DataFrame({
    'Modelo': ['k-NN Classifier', 'MLP Classifier', 'k-NN Regressor', 'MLP Regressor'],
    'Accuracy': [accuracy_score(y_test, y_pred_knn), accuracy_score(y_test, y_pred_mlp), None, None],
    'F1': [f1_score(y_test, y_pred_knn), f1_score(y_test, y_pred_mlp), None, None],
    'AUC-ROC': [roc_auc_score(y_test, y_pred_knn), roc_auc_score(y_test, y_pred_mlp), None, None],
    'RMSE': [None, None, mean_squared_error(y_reg_test, y_pred_knn_reg, squared=False), mean_squared_error(y_reg_test, y_pred_mlp_reg, squared=False)],
    'MAE': [None, None, mean_absolute_error(y_reg_test, y_pred_knn_reg), mean_absolute_error(y_reg_test, y_pred_mlp_reg)],
    'R2': [None, None, r2_score(y_reg_test, y_pred_knn_reg), r2_score(y_reg_test, y_pred_mlp_reg)]
})
display(results)

## 7. Exportación a HTML
Puedes exportar el notebook a HTML desde el menú de Jupyter/VS Code: `File > Export As > HTML`.

## 8. Monitoreo del Entrenamiento y Curvas de Aprendizaje
Para visualizar el entrenamiento de las redes neuronales (MLP), puedes graficar la evolución de la función de pérdida (loss) y la accuracy por época. Esto te permite identificar overfitting, underfitting y el comportamiento del aprendizaje.

A continuación se muestra cómo obtener y graficar el historial de loss y accuracy por época para el mejor modelo MLP:

In [ ]:
# Graficar la evolución del loss y accuracy por época para el mejor MLPClassifier
import matplotlib.pyplot as plt
import numpy as np

# Entrenar un MLPClassifier con warm_start=True para obtener el historial de loss
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1, warm_start=True, random_state=42, verbose=False)
n_epochs = 50
train_loss = []
train_acc = []
for epoch in range(n_epochs):
    mlp.fit(X_train, y_train)
    train_loss.append(mlp.loss_)
    train_acc.append(mlp.score(X_train, y_train))

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(range(1, n_epochs+1), train_loss, marker='o')
plt.title('MLPClassifier - Loss por Época')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), train_acc, marker='o', color='green')
plt.title('MLPClassifier - Accuracy por Época')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

## 9. Prueba del Modelo con Ejemplos Reales
Puedes probar el modelo con nuevos datos reales o ejemplos personalizados. Solo debes asegurarte de que el input tenga el mismo formato y preprocesamiento que los datos de entrenamiento (mismas columnas, escalado, reducción de dimensionalidad si aplica).
A continuación se muestra cómo hacer una predicción con un ejemplo real o nuevo paciente:

In [ ]:
# Ejemplo: predicción con un nuevo paciente (input real o simulado)
# Supón que tienes un nuevo paciente con los siguientes valores (ajusta según tus features):
nuevo_paciente = np.array([[0.5, 1.2, -0.3, 0.8, 1.1]])  # Reemplaza con tus valores y dimensiones reales

# Si usaste reducción de dimensionalidad (PCA, BNMF, NMF), transforma el input igual que en entrenamiento
# Por ejemplo, si usaste scaler y pca:
# nuevo_paciente_scaled = scaler.transform(nuevo_paciente)
# nuevo_paciente_latente = pca.transform(nuevo_paciente_scaled)
# pred = mlp_clf.predict(nuevo_paciente_latente)

# Si entrenaste directamente sobre features originales o embeddings latentes:
pred = mlp_clf.predict(nuevo_paciente)
print('Predicción de clase (MLP):', pred[0])

## 10. Pruebas de Integración: Flujo Completo desde EDA y BNMF/Clustering
En esta sección puedes probar el modelo usando datos generados o transformados en los cuadernos anteriores (EDA y Reducción/BNMF/Clustering).
Esto asegura que todo el pipeline funciona de extremo a extremo, desde la limpieza y reducción de dimensionalidad hasta la predicción clínica.

A continuación se muestra cómo tomar un paciente del dataset original, aplicar el preprocesamiento y obtener la predicción final:

In [ ]:
# Selecciona un paciente del dataset original (por ejemplo, el primero del test set EDA)
# Aplica el mismo preprocesamiento que en los cuadernos anteriores
# (scaler, PCA, BNMF, etc.) para obtener el embedding latente y predecir

# Ejemplo:
paciente_original = df.iloc[[0]].drop(columns=['hospital_expire_flag', 'data_split'], errors='ignore')

# Si usaste scaler y pca:
# paciente_scaled = scaler.transform(paciente_original)
# paciente_latente = pca.transform(paciente_scaled)
# pred = mlp_clf.predict(paciente_latente)

# Si usaste BNMF/NMF y tienes el embedding W:
# idx = 0  # índice del paciente en W_bnmf o W_nmf
# paciente_latente = W_bnmf[idx].reshape(1, -1) if W_bnmf is not None else W_nmf[idx].reshape(1, -1)
# pred = mlp_clf.predict(paciente_latente)

print('Predicción de clase para paciente original:', pred[0])